Nousallons travailler dans un envirronnement python dans lequel nous allons intégrer du sqlite

In [3]:
import numpy as np
import pandas as pd
from google.colab import files
uploaded = files.upload()


Saving netflix_titles.csv to netflix_titles (1).csv


In [6]:
df_netflix = pd.read_csv('netflix_titles.csv')
df_netflix.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


## Maintenant nous importons la bibliothèque nécessaire pour interagir avec une base de données SQL. et nous créeons un moteur SQLite en mémoire pour stocker ces données temporairement. et nous exportons le DataFrame df_netflix dans une table nommée netflix de la base de données SQLite.

In [7]:
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)
# exporter la dataframe comme un tableau ver notre sql engine
df_netflix.to_sql("netflix", con =engine)

8807

Petit test sur le code

1. *Afficher les 5 premiers éléments du tableau *texte en italique*

In [8]:
sql='''

Select * from netflix
limit 5

   '''
df_sql = pd.read_sql_query(sql,con=engine)
df_sql.head()

,index,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,None,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,1,s2,TV Show,Blood & Water,None,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",None,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,3,s4,TV Show,Jailbirds New Orleans,None,None,None,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,4,s5,TV Show,Kota Factory,None,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


2. *Quelles sont les valeurs distinctes de l'identifiant de contenu (`show_id`) dans la base de données Netflix ?*

In [11]:
sql='''
select distinct show_id
from netflix
    '''
df_sql = pd.read_sql_query(sql,con=engine)
df_sql.head()

,show_id
0,s1
1,s2
2,s3
3,s4
4,s5


3. *Quels sont les titres uniques de contenus ajoutés après l'année 2020 sur Netflix ?*

In [12]:
sql='''
select distinct title
from netflix
where release_year > 2020
    '''
df_sql = pd.read_sql_query(sql,con=engine)
df_sql.head()

,title
0,Blood & Water
1,Ganglands
2,Jailbirds New Orleans
3,Kota Factory
4,Midnight Mass


4. *Quel est le titre unique de contenu parmi ceux sortis après 2020 et intitulés "Blood & Water" ou avant 2021 pour "Kota Factory", mais qui n'ont pas été produits en Inde ?*

In [19]:
sql='''
select title from netflix
where (release_year>2020 AND title="Blood & Water") or (release_year<2021 and title="Kota Factory" and not country ="India")
    '''
df_sql = pd.read_sql_query(sql,con=engine)
df_sql.head()

,title
0,Blood & Water


5. *Quels sont les titres uniques des contenus Netflix, triés par ordre alphabétique ?*

In [20]:
sql='''
select distinct title from netflix
order by title asc
    '''
df_sql = pd.read_sql_query(sql,con=engine)
df_sql.head()

,title
0,#Alive
1,#AnneFrank - Parallel Stories
2,#FriendButMarried
3,#FriendButMarried 2
4,#Roxy


6. *Quels sont les enregistrements qui ont PG-13 comme rating*

In [22]:
sql='''
select * from netflix
where rating="PG-13"
    '''
df_sql = pd.read_sql_query(sql,con=engine)
df_sql.head()

,index,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,None,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,9,s10,Movie,The Starling,Theodore Melfi,"Melissa McCarthy, Chris O'Dowd, Kevin Kline, T...",United States,"September 24, 2021",2021,PG-13,104 min,"Comedies, Dramas",A woman adjusting to life after a loss contend...
2,27,s28,Movie,Grown Ups,Dennis Dugan,"Adam Sandler, Kevin James, Chris Rock, David S...",United States,"September 20, 2021",2010,PG-13,103 min,Comedies,Mourning the loss of their beloved junior high...
3,28,s29,Movie,Dark Skies,Scott Stewart,"Keri Russell, Josh Hamilton, J.K. Simmons, Dak...",United States,"September 19, 2021",2013,PG-13,97 min,"Horror Movies, Sci-Fi & Fantasy",A family’s idyllic suburban life shatters when...
4,29,s30,Movie,Paranoia,Robert Luketic,"Liam Hemsworth, Gary Oldman, Amber Heard, Harr...","United States, India, France","September 19, 2021",2013,PG-13,106 min,Thrillers,"Blackmailed by his company's CEO, a low-level ..."


7.*Quelle est l'année de sortie la plus ancienne et la plus récente des contenus, le nombre d'années de sortie distinctes, et la moyenne des années de sortie de tous les contenus ?*

In [23]:
sql='''
Select min(release_year) as min_year, max(release_year) as max_year, count(distinct(release_year)) as distinct_year, avg(release_year) as avg_year
from netflix
    '''
df_sql = pd.read_sql_query(sql,con=engine)
df_sql.head()

,min_year,max_year,distinct_year,avg_year
0,1925,2021,74,2014.180198


8. *Quels pays ont un nom qui finit par "ia", commence par "ia", contient "ia" ou commence et se termine par "a" ?*

In [42]:
sql='''
select distinct country as value from netflix where country like "%ia"
union all
select distinct country as value from netflix where country like "ia%"
union all
select distinct country as value from netflix where country like "%ia%"
union all
select distinct country as value from netflix where country like "a%a%"

    '''
df_sql = pd.read_sql_query(sql,con=engine)
df_sql.head()

,value
0,India
1,"United States, Ghana, Burkina Faso, United Kin..."
2,Australia
3,Nigeria
4,Russia


9. *Quels sont les contenus produits en Inde, aux États-Unis ou en Australie, triés par pays en ordre croissant ?*

In [38]:
sql='''
select * from netflix
where country="India" or country="United States" or country="Australia"
order by country asc
    '''
df_sql = pd.read_sql_query(sql,con=engine)
df_sql

,index,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,25,s26,TV Show,Love on the Spectrum,None,Brooke Satchwell,Australia,"September 21, 2021",2021,TV-14,2 Seasons,"Docuseries, International TV Shows, Reality TV",Finding love can be hard for anyone. For young...
1,108,s109,TV Show,Dive Club,None,"Aubri Ibrag, Sana'a Shaik, Miah Madden, Mercy ...",Australia,"September 3, 2021",2021,TV-G,1 Season,"Kids' TV, TV Dramas, Teen TV Shows","On the shores of Cape Mercy, a skillful group ..."
2,120,s121,TV Show,Heroes of Goo Jit Zu,None,"Jon Allen, Kellen Goff, Joe Hernandez, Kaiji Tang",Australia,"September 2, 2021",2021,TV-Y7,1 Season,"Kids' TV, TV Comedies","After a meteor crash, a group of zoo animals t..."
3,393,s394,Movie,A Second Chance: Rivals!,Clay Glen,"Emily Morris, Stella Shute, Eva Grados, India ...",Australia,"July 23, 2021",2021,PG,91 min,"Children & Family Movies, Sports Movies",Crushed when she doesn't qualify for the Olymp...
4,506,s507,Movie,This Little Love Of Mine,Christine Luby,"Saskia Hampele, Liam McIntyre, Lynn Gilmartin,...",Australia,"July 7, 2021",2021,TV-G,92 min,"International Movies, Romantic Movies",A workaholic lawyer returns to her island home...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3872,8791,s8792,Movie,Young Adult,Jason Reitman,"Charlize Theron, Patton Oswalt, Patrick Wilson...",United States,"November 20, 2019",2011,R,94 min,"Comedies, Dramas, Independent Movies",When a divorced writer gets a letter from an o...
3873,8793,s8794,Movie,"Yours, Mine and Ours",Raja Gosnell,"Dennis Quaid, Rene Russo, Sean Faris, Katija P...",United States,"November 20, 2019",2005,PG,88 min,"Children & Family Movies, Comedies",When a father of eight and a mother of 10 prep...
3874,8802,s8803,Movie,Zodiac,David Fincher,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",United States,"November 20, 2019",2007,R,158 min,"Cult Movies, Dramas, Thrillers","A political cartoonist, a crime reporter and a..."
3875,8804,s8805,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,"November 1, 2019",2009,R,88 min,"Comedies, Horror Movies",Looking to survive in a world taken over by zo...


10. *Quels contenus sont sortis entre 2020 et 2021 ?*

In [41]:
sql='''
select * from netflix
where release_year between 2020 and 2021
    '''
df_sql = pd.read_sql_query(sql,con=engine)
df_sql.head()

,index,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,None,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,1,s2,TV Show,Blood & Water,None,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",None,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,3,s4,TV Show,Jailbirds New Orleans,None,None,None,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,4,s5,TV Show,Kota Factory,None,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


11. *Quels sont les identifiants (`show_id`) correspondant aux deux tables de contenu avec le type et le titre, si les identifiants (`show_id`) sont partagés dans les deux tables de Netflix ?*

In [43]:
sql='''
select
n1.show_id as Show_idtable1,
n2.type as Show_idTable2,
n2.title as Showtable2
from netflix n1
join netflix n2 on n1.show_id = n2.show_id

    '''
df_sql = pd.read_sql_query(sql,con=engine)
df_sql.head()

,Show_idtable1,Show_idTable2,Showtable2
0,s1,Movie,Dick Johnson Is Dead
1,s2,TV Show,Blood & Water
2,s3,TV Show,Ganglands
3,s4,TV Show,Jailbirds New Orleans
4,s5,TV Show,Kota Factory


12. *Quels sont les contenus Netflix provenant soit d'Inde, soit des États-Unis ?*

In [44]:
sql='''
select * from netflix where country='India'
union all
select * from netflix where country='United States'

    '''
df_sql = pd.read_sql_query(sql,con=engine)
df_sql.head()

,index,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,4,s5,TV Show,Kota Factory,None,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...
1,24,s25,Movie,Jeans,S. Shankar,"Prashanth, Aishwarya Rai Bachchan, Sri Lakshmi...",India,"September 21, 2021",1998,TV-14,166 min,"Comedies, International Movies, Romantic Movies",When the father of the man she loves insists t...
2,39,s40,TV Show,Chhota Bheem,None,"Vatsal Dubey, Julie Tejwani, Rupa Bhimani, Jig...",India,"September 16, 2021",2021,TV-Y7,3 Seasons,Kids' TV,"A brave, energetic little boy with superhuman ..."
3,50,s51,TV Show,Dharmakshetra,None,"Kashmira Irani, Chandan Anand, Dinesh Mehta, A...",India,"September 15, 2021",2014,TV-PG,1 Season,"International TV Shows, TV Dramas, TV Sci-Fi &...","After the ancient Great War, the god Chitragup..."
4,66,s67,TV Show,Raja Rasoi Aur Anya Kahaniyan,None,None,India,"September 15, 2021",2014,TV-G,1 Season,"Docuseries, International TV Shows",Explore the history and flavors of regional In...


13. *Combien de contenus sont produits en Inde, aux États-Unis et en Afrique du Sud dans la base de données Netflix ?*

In [49]:
sql='''
SELECT
SUM(CASE WHEN country='India' THEN 1 else 0 end) as sum_india,
SUM(CASE WHEN country="United States" THEN 1 else 0 end) as sum_usa,
SUM(CASE WHEN country="South Africa" THEN 1 else 0 end) as sum_southafrica
FROM netflix

    '''
df_sql = pd.read_sql_query(sql,con=engine)
df_sql.head()

,sum_india,sum_usa,sum_southafrica
0,972,2818,30


14. Quels sont les titres uniques de contenus dans la table Netflix (en utilisant une sous-requête) ?

In [50]:
sql='''
select distinct(a.title) from (select * from netflix) a
    '''
df_sql = pd.read_sql_query(sql,con=engine)
df_sql.head()

,title
0,Dick Johnson Is Dead
1,Blood & Water
2,Ganglands
3,Jailbirds New Orleans
4,Kota Factory


15. *Quelle est la première valeur d'année de sortie non nulle dans la table Netflix, ou 0 si elle n'existe pas ?*

In [51]:
sql='''
select coalesce(release_year,0) as first_non_zero_value from netflix
limit 1
    '''
df_sql = pd.read_sql_query(sql,con=engine)
df_sql.head()

,first_non_zero_value
0,2020


16. *Quelle est la première valeur d'année de sortie convertie en valeur flottante dans la table Netflix ?*

In [52]:
sql='''
select cast(release_year as float) as first_non_zero_value from netflix
    '''
df_sql = pd.read_sql_query(sql,con=engine)
df_sql.head()

,first_non_zero_value
0,2020.0
1,2021.0
2,2021.0
3,2021.0
4,2021.0


17. *Quels contenus ont des années de sortie consécutives (en "back-to-back") et quels titres sont concernés, triés par ordre croissant des années de sortie ?*

In [53]:
sql='''
select
title,
release_year,
case when release_year=lag(release_year) over (order by release_year asc) then 1 else 0 end as Back_to_Back
from netflix
order by release_year asc

 '''
df_sql = pd.read_sql_query(sql,con=engine)
df_sql.head()

,title,release_year,Back_to_Back
0,Pioneers: First Women Filmmakers*,1925,0
1,Prelude to War,1942,0
2,The Battle of Midway,1942,1
3,Undercover: How to Operate Behind Enemy Lines,1943,0
4,Why We Fight: The Battle of Russia,1943,1


18. *Quels sont les titres, types et réalisateurs de chaque contenu, et quel est leur numéro de rang lorsqu'on trie les titres par ordre croissant ?*

In [54]:
sql='''
select
title,type,director,
ROW_NUMBER() OVER (ORDER BY
        title asc) as row_number
        from netflix
group by title
order by title asc


 '''
df_sql = pd.read_sql_query(sql,con=engine)
df_sql.head()

,title,type,director,row_number
0,#Alive,Movie,Cho Il,1
1,#AnneFrank - Parallel Stories,Movie,"Sabina Fedeli, Anna Migotto",2
2,#FriendButMarried,Movie,Rako Prijanto,3
3,#FriendButMarried 2,Movie,Rako Prijanto,4
4,#Roxy,Movie,Michael Kennedy,5


19. Quel est le premier pays dont le nom finit par "ia", le premier pays dont le nom commence par "a", et le premier pays contenant "a" avec une longueur minimale de 4 caractères ?

In [55]:
sql='''
select a.value from (
Select
distinct ' country that End with ia :- '|| country as Value,
DENSE_RANK() OVER (ORDER BY country asc) as rank from netflix
where country like"%ia"
)a
where rank=1

union all

select a.value from (
Select
distinct ' first country that starts with a :- '|| country as Value,
DENSE_RANK() OVER (ORDER BY country asc) as rank from netflix
where country like"a%"
)a
where rank=1

union all

select a.value from (
Select
distinct ' country that has a and minimum length 4 :- '|| country as Value,
DENSE_RANK() OVER (ORDER BY country asc) as rank from netflix
where country like"a__%"
)a
where rank=1

 '''
df_sql = pd.read_sql_query(sql,con=engine)
df_sql.head()

,Value
0,"country that End with ia :- , France, Algeria"
1,first country that starts with a :- Argentina
2,country that has a and minimum length 4 :- Ar...


20. Quel est le pays ayant le plus grand nombre de contenus uniques sur Netflix ?

In [57]:
sql='''

with b as (
select
a.country,
a.count_of_shows,
row_number () over (order by a.count_of_shows desc) as rank
from
(
select
count(distinct title) as count_of_shows,
country
from netflix
group by country
) a
)
select country as country_most_shows from b
where  b.rank=1

 '''
df_sql = pd.read_sql_query(sql,con=engine)
df_sql.head()

,country_most_shows
0,United States
